In [2]:
import math
import urllib.request
import numpy as np

In [3]:
data_url = 'http://www.rit-mcsl.org/MunsellRenotation/all.dat'
data_file = 'all.dat'

## A bunch of Functions

Here are just a bunch of fuctions I found in the source code for the site:

In [30]:
def fetch(url = data_url, fname = data_file, clobber = False):
    # If we're not clobbering the file, quit early if we can read it.
    if not clobber:
        try:
            with open(fname) as f:
                return
        except FileNotFoundError:
            pass
    # Fetch the data from the internet.
    data = urllib.request.urlopen(url)
    with open(fname, 'w') as f:
        f.write(data.read().decode())

def load(fname = data_file):
    with open(fname) as f:
        return [line for line in f]
    
hues = ['R', 'YR', 'Y', 'GY', 'G', 'BG', 'B', 'PB', 'P', 'RP']
hues = dict((v, 10 * k) for k, v in enumerate(hues))

def parse(lines):
    def parse_one(line):
        ans = line.split()
        ans[0] = ans[0] + ' ' + ans[1] + '/' + ans[2]
        for i, v in enumerate(ans[3:]):
            ans[i + 1] = float(v)
        return ans[:4]
    return [parse_one(line) for line in lines[1:]]

def xyY2XYZ(xyY):
    x, y, Y = xyY
    if abs(y) < 1e-100:
        y = 1e-100
    X = x*Y/y
    Z = (1-x-y)*Y/y
    return [X, Y, Z]

In [35]:
min(9,8)

8

In [38]:
def XYZ2RGB(XYZ):
    XYZ = np.array(XYZ)
    M = np.array([[3.2404542, -1.5371385, -0.4985314],
                  [-0.9692660,  1.8760108,  0.0415560],
                  [0.0556434, -0.2040259,  1.0572252]])
    rgb = M @ XYZ
    
    def gamma(u):
        if u <= 0.003138:
            return max(12.92*u,0)
        else:
            return min(1.055*u**0.416 - 0.055,1)
    
    return [round(255*gamma(x)) for x in rgb]

In [28]:
    
    






def xyy_to_rgb_linear(x, y, y2):
    if abs(y) < 1e-100:
        y = 1e-100
    y2 /= 100
    x2 = y2 * x / y
    z2 = y2 * (1 - x - y) / y
    return (3.2303 * x2 - 1.5877 * y2 - 0.4909 * z2,
        -0.9834 * x2 + 1.9125 * y2 + 0.0440 * z2,
        0.0539 * x2 - 0.2004 * y2 + 0.9706 * z2)

def rgb_delinearize(*rgb):
    ans = [0, 0, 0]
    for i, c in enumerate(rgb):
        if c <= 0.0031308:
            ans[i] = 12.92 * c
        else:
            ans[i] = 1.055 * math.pow(c, 1/2.4) - 0.055
    return ans

def hls_to_hlc(h, l, s):
    return h, l, s * (1 - abs(2 * l - 1))

def average_triplets(triplets, ignore_nones = False):
    ans = [None, None, None]
    for i in range(3):
        filtered_values = list(filter(lambda x: x is not None, [t[i] for t in triplets]))
        num_values = len(filtered_values)
        if ((num_values != len(triplets)) and not ignore_nones) or num_values < 1:
            continue
        ans[i] = sum(filtered_values) / len(filtered_values)
    return ans

def value_to_y(v):
    return v * (1.1914 + v * (-0.22533 + v * (0.23352 + v * (-0.020484 + v * 0.00081939))))

def process():
    entries = parse(load())
    for entry in entries:
        entry[-1] = 0.975 * entry[-1]
        entry[-3:] = rgb_delinearize(*xyy_to_rgb_linear(*entry[-3:]))
    # Fill the table that our javascript will read.
    c = [[[[None,None,None] for k in range(27)] for j in range(16)] for i in range(40)]
    for entry in entries:
        c[round(entry[0] * 0.4 - 1)][
            round((entry[1] > 1) and (entry[1] + 4) or (entry[1] * 5))
            ][round(entry[2]/2)] = entry[-3:]
    # Insert grays (chroma = 0) by the ASTM Standard
    value_list = [x * 0.2 for x in range(0, 5)] + list(range(1, 11))
    for j in range(15):
        gray = rgb_delinearize(*xyy_to_rgb_linear(0.31006, 0.31616, value_to_y(value_list[j])))
        for i in range(40):
            c[i][j][0] = gray
    return c

def prettify(c):
    return '[\n%s\n]' % ',\n'.join([
        '[\n%s\n]' % ',\n'.join([
            ' [%s]' % ','.join([
                '[%s]' % ','.join([
                    ((w is None) and 'NaN' or ('%.5f' % w)) for w in z])
                for z in y])
            for y in x])
        for x in c])

def adj(C):
    if abs(C) < 0.0031308:
        return 12.92 * C
    return 1.055 * abs(C)**0.41666 - 0.055

SyntaxError: invalid syntax (<ipython-input-28-3c9bb1a43f20>, line 38)

In [5]:
fetch(url = data_url, fname = data_file, clobber = False)


## Processing an individual

In [8]:
Table = load()

table = Table[1:]
table

['    2.5GY        0.2         2     0.713     1.414   0.237\n',
 '      5GY        0.2         2     0.449     1.145   0.237\n',
 '    7.5GY        0.2         2     0.262     0.837   0.237\n',
 '    7.5GY        0.2         4    -0.078      2.16   0.237\n',
 '     10GY        0.2         2     0.185     0.676   0.237\n',
 '     10GY        0.2         4    -0.257     1.233   0.237\n',
 '     2.5G        0.2         2     0.144     0.584   0.237\n',
 '     2.5G        0.2         4    -0.235     0.891   0.237\n',
 '       5G        0.2         2     0.117     0.516   0.237\n',
 '       5G        0.2         4    -0.209     0.719   0.237\n',
 '     7.5G        0.2         2     0.097     0.458   0.237\n',
 '     7.5G        0.2         4    -0.181     0.575   0.237\n',
 '      10G        0.2         2      0.08     0.397   0.237\n',
 '      10G        0.2         4    -0.137     0.425   0.237\n',
 '    2.5BG        0.2         2     0.068     0.332   0.237\n',
 '    2.5BG        0.2   

In [21]:
spl = table[1].split()
Mun_name = spl[0] + ' ' + spl[1] + '/' + spl[2]

In [27]:
xyY = [float(spl[3]), float(spl[4]), float(spl[5])]

XYZ = [xyY[0]*xyY[2]/xyY[1], xyY[2], (1 - xyY[0] - xyY[1])*xyY[2]/xyY[1]]


In [40]:
xyY = [float(spl[3]), float(spl[4]), float(spl[5])]
x, y, Y = xyY
XYZ = xyY2XYZ(xyY)
rgb = XYZ2RGB(XYZ)
rgb

[0, 160.0, 0]

In [34]:
M = np.array([[3.2404542, -1.5371385, -0.4985314],
               [-0.9692660,  1.8760108,  0.0415560],
               [0.0556434, -0.2040259,  1.0572252]])
rgb = M @ XYZ
rgb

array([-0.00184881,  0.34942445, -0.17316887])

In [20]:
rgb = xyy_to_rgb_linear(xyY[0],xyY[1],xyY[2])
rgb

NameError: name 'xyY' is not defined

In [19]:
x,y,Y = rgb_delinearize(*rgb)

NameError: name 'rgb' is not defined

In [32]:
New_Table = []
Table = load()
table = Table[1:]

for row in table:
    spl = row.split()
    Mun_name = spl[0] + ' ' + spl[1] + '/' + spl[2]

    xyY = [float(spl[3]), float(spl[4]), float(spl[5])]
    xyY[-1] = 0.975*xyY[-1]
    xyY = rgb_delinearize(*xyy_to_rgb_linear(*xyY))

In [17]:
entries = parse(load())

In [18]:
entries

[['2.5GY 0.2/2', 0.713, 1.414, 0.237],
 ['5GY 0.2/2', 0.449, 1.145, 0.237],
 ['7.5GY 0.2/2', 0.262, 0.837, 0.237],
 ['7.5GY 0.2/4', -0.078, 2.16, 0.237],
 ['10GY 0.2/2', 0.185, 0.676, 0.237],
 ['10GY 0.2/4', -0.257, 1.233, 0.237],
 ['2.5G 0.2/2', 0.144, 0.584, 0.237],
 ['2.5G 0.2/4', -0.235, 0.891, 0.237],
 ['5G 0.2/2', 0.117, 0.516, 0.237],
 ['5G 0.2/4', -0.209, 0.719, 0.237],
 ['7.5G 0.2/2', 0.097, 0.458, 0.237],
 ['7.5G 0.2/4', -0.181, 0.575, 0.237],
 ['10G 0.2/2', 0.08, 0.397, 0.237],
 ['10G 0.2/4', -0.137, 0.425, 0.237],
 ['2.5BG 0.2/2', 0.068, 0.332, 0.237],
 ['2.5BG 0.2/4', -0.1, 0.325, 0.237],
 ['5BG 0.2/2', 0.066, 0.261, 0.237],
 ['5BG 0.2/4', -0.048, 0.223, 0.237],
 ['7.5BG 0.2/2', 0.072, 0.226, 0.237],
 ['7.5BG 0.2/4', -0.016, 0.181, 0.237],
 ['10BG 0.2/2', 0.085, 0.195, 0.237],
 ['10BG 0.2/4', 0.015, 0.148, 0.237],
 ['2.5B 0.2/2', 0.097, 0.177, 0.237],
 ['2.5B 0.2/4', 0.039, 0.128, 0.237],
 ['5B 0.2/2', 0.111, 0.164, 0.237],
 ['5B 0.2/4', 0.063, 0.114, 0.237],
 ['7.5B 0.2/2